# **serve** with `tornado`

In [ ]:
from tornado import web
app = web.Application()
app.listen(8899);

## **handle** a request

In [ ]:
from fib import fib_typed
class FibHandler(web.RequestHandler):
    def get(self, n):
        self.write(str(fib_typed(int(n))))
app.add_handlers(".*", [
    (r"/fib/(?P<n>\d*)", FibHandler)
])

## **request** with a client

In [ ]:
from tornado.httpclient import AsyncHTTPClient 
r = await AsyncHTTPClient().fetch(
    "http://127.0.0.1:8899/fib/10"
)
r.body

# **acceptance** testing with `robotframework`

# **conformance** testing with `schemathesis`

# load testing with `locust`